# Read stim_time from raw XDAQ file

In [ ]:
import os
from pathlib import Path
import numpy as np
import allego_file_reader as afr # remember to copy this .py file to the folder
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
sample_rate = 30000 # true ephys sample rate, get from 01_ephys_sychronizaiton_XDAQ.ipynb
lag_time = 0 # time that ephys start later than camera, get from 01_ephys_sychronizaiton_XDAQ.ipynb
stim_threshhold = 500 # in uV
stim_channel_number = 15 # choose a stimulation channel to find stim_time 
time_start = None #
time_end = None # start and end time (seconds) to read ephys data, can default as 'None'

In [ ]:
#if read from recording channel: # add stim info to record all the pulse if stim is a train
stim_train = False # True = a pulse train, False = single pulse
stim_gap = 0 # if a train time gap between each pulse, in second
stim_num = 0 # if a train how many pulse are there in a train

In [ ]:
target_dir = "ephys"
p = Path(target_dir).expanduser()
all_xdat_datasource_names = [Path(elem.stem).stem for elem in list(p.glob('**/*xdat.json'))]
all_xdat_datasource_names[0]
target_file = str(Path(target_dir, all_xdat_datasource_names[0]))
time_range = afr.get_allego_xdat_time_range(target_file)
metadata = afr.read_allego_xdat_metadata(target_file)

channel, timestamps, time_samples = afr.read_allego_xdat_all_signals(target_file, 
                                                                          time_start, time_end)

# to read stim: either use a recording channel or read from STIM channel
# there are 39 channels: recording channel 1~32, then AI1, AI2, DI1, DI2, STIM, DO1, DO2
# eg. recordnig channel 15 is channel[14], AI1 is channel[32], STIM is channel[36]

Load stim channel for stim_time and to see current leakage

In [ ]:
# calculate stim_time
stim_channel = channel[stim_channel_number]
stim_time = []
skip = 0
for index in range(0,len(timestamps)-1):
    if (skip>0):
        skip = skip - 1
        continue
    if ((stim_channel[index+1] - stim_channel[index] > stim_threshhold)) or (stim_channel[index+1] - stim_channel[index] < (-stim_threshhold)):
        skip = sample_rate
        stim_time.append(index)

stim_time = [element / sample_rate + lag_time for element in stim_time]
print(stim_time)

In [ ]:
# add if stim is a train
if (stim_train):
    stim_time = [element + count * stim_gap for element in stim_time for count in range(stim_num)]

In [ ]:
# save each stim info in a .npy file in a new folder

# Create a new folder
folder_name = "stim-time"
os.makedirs(folder_name, exist_ok=True)

# Save the array to an .npy file inside the folder
file_path = os.path.join(folder_name, 'stim-time.npy')
np.save(file_path, stim_time)

In [ ]:
# to plot all the channels
time = np.arange(0,len(timestamps))
fig, axs = plt.subplots(8,4,figsize=(80,40))
for index in range(32):
    i=int(index/4)
    j=index%4
    axs[i,j].plot(time,channel[index])
    plt.title("channel:{}".format(index))
plt.subplots_adjust(wspace=0.3,hspace=0.22)
plt.show()